In [113]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from scipy.sparse import issparse, vstack, hstack, csr_matrix as sparse
%matplotlib inline

In [114]:
%%HTML
<style type="text/css">
table, td{
    border-style: solid;
}
th {
  background-color: #4CAF50;
  color: white;
}
tr:nth-child(even) {background-color: #f0f0f0;}
tr:hover {background-color: #4CAF50;}

In [115]:
from pypower.api import case30pwl, case30Q 
from pypower.api import case30, ppoption, runpf, printpf, runopf
from pypower.loadcase import loadcase

from drawings import *
from utils import *
from constr_setting import *
from pypower.api import opf_consfcn, opf_costfcn
from pypower.idx_bus import BUS_TYPE, REF, VM, VA, MU_VMAX, MU_VMIN, LAM_P, LAM_Q
from scipy.optimize import minimize, nnls, linprog

In [116]:
bus_cols, gen_cols, brch_cols, cost_cols = get_cols()

In [117]:
ppc = case30()

ppopt = ppoption(PF_ALG=1, RETURN_RAW_DER=True, OPF_FLOW_LIM=1)
#ppopt

In [118]:
pd.DataFrame(ppc['bus'], columns=bus_cols[: LAM_P])

,BUS_I,BUS_TYPE,PD,QD,GS,BS,BUS_AREA,VM,VA,BASE_KV,ZONE,VMAX,VMIN
0,1.0,3.0,0.0,0.0,0.0,0.00,1.0,1.0,0.0,135.0,1.0,1.05,0.95
1,2.0,2.0,21.7,12.7,0.0,0.00,1.0,1.0,0.0,135.0,1.0,1.10,0.95
2,3.0,1.0,2.4,1.2,0.0,0.00,1.0,1.0,0.0,135.0,1.0,1.05,0.95
3,4.0,1.0,7.6,1.6,0.0,0.00,1.0,1.0,0.0,135.0,1.0,1.05,0.95
4,5.0,1.0,0.0,0.0,0.0,0.19,1.0,1.0,0.0,135.0,1.0,1.05,0.95
5,6.0,1.0,0.0,0.0,0.0,0.00,1.0,1.0,0.0,135.0,1.0,1.05,0.95
6,7.0,1.0,22.8,10.9,0.0,0.00,1.0,1.0,0.0,135.0,1.0,1.05,0.95
7,8.0,1.0,30.0,30.0,0.0,0.00,1.0,1.0,0.0,135.0,1.0,1.05,0.95
8,9.0,1.0,0.0,0.0,0.0,0.00,1.0,1.0,0.0,135.0,1.0,1.05,0.95
9,10.0,1.0,5.8,2.0,0.0,0.00,3.0,1.0,0.0,135.0,1.0,1.05,0.95


In [119]:
#pd.DataFrame(ppc['gen'], columns=gen_cols[: MU_PMAX])

In [120]:
gens = ppc['gen'][:, 0]

load_list = []
bids = [6, 7, 15, 19, 29]
for bus_str in ppc['bus']:
    tmp = np.zeros(21)
    if bus_str[0] not in gens:
        if bus_str[0] not in gens and (bus_str[0] in bids):
            #print('load #%d' % bus_str[0])
            tmp[0] = bus_str[0]

            tmp[1] = -30.#bus_str[2]
            tmp[2] = -15.#bus_str[3]

            tmp[3] = -0.0001
            tmp[4] = -15.

            tmp[5] = bus_str[7]
            tmp[6] = 100.

            tmp[7] = 1.

            #fixed here
            tmp[8] = -0.0001
            tmp[9] = -30.

            bus_str[2], bus_str[3] = 0., 0.
            load_list.append(tmp)
ppc['gen'] = np.concatenate((ppc['gen'], np.array(load_list)))

In [121]:
#pd.DataFrame(ppc['gen'], columns=gen_cols[: MU_PMAX])

In [122]:
#pd.DataFrame(ppc['branch'], columns=brch_cols[: PF])

In [123]:
alpha = 1.01
#bound max flow 3 + 1 <-> 11 + 1
ppc['branch'][14][5] *= alpha
#bound max flow 9 + 1 <-> 16 + 1
ppc['branch'][25][5] *= alpha
#bound max flow 27 <-> 26
#ppc['branch'][35][5] *= alpha
#bound max flow 18 <-> 19
ppc['branch'][23][5] *= alpha
#bound max flow 22 <-> 23
ppc['branch'][31][5] *= alpha

In [124]:
#pd.DataFrame(ppc['branch'], columns=brch_cols[: PF])

In [125]:
#results['nln']['mu']['u'].keys(), results['nln']['mu']['l'].keys()

In [126]:
#results['om']

## Data preparation to fulfil the task

Bus data:

<table>
    <tr>
            <td> $bus_i$ </td>
            <td> type </td>
            <td> $P_d$ </td>
            <td> $Q_d$ </td>
            <td> $G_s$ </td>
            <td> $B_s$ </td>
            <td> area </td>
            <td> $V_m$ </td>
            <td> $V_a$ </td>
            <td> base~KV </td>
            <td> zone </td>
            <td> $V_{max}$ </td>
            <td> $V_{min}$ </td>
    </tr>
</table>

Generator data:

<table>
    <tr>
            <td> $bus$ </td>
            <td> $P_g$ </td>
            <td> $Q_g$ </td>
            <td> $Q_{max}$ </td>
            <td> $Q_{min}$ </td>
            <td> mBase </td>
            <td> status </td>
            <td> $P_{max}$ </td>
            <td> $P_{min}$ </td>
            <td> $\ldots$ </td>
    </tr>
</table>

Generator cost data has the following structure:

<table>
    <tr>
            <td> 1 </td>
            <td> startup </td>
            <td> shutdown </td>
            <td> n </td>
            <td> $x_1$ </td>
            <td> $y_1$ </td>
            <td> $\ldots$ </td>
            <td> $x_n$ </td>
            <td> $y_n$ </td>
    </tr>
    <tr>
            <td> 2 </td>
            <td> startup </td>
            <td> shutdown </td>
            <td> n </td>
            <td> $c_{n - 1}$ </td>
            <td> $\ldots$ </td>
            <td> $\ldots$ </td>
            <td> $\ldots$ </td>
            <td> $c_0$ </td>
    </tr>
</table>

Convert cost function to linear changing the order of polynom to 1

In [127]:
ppc['gencost'][:,3] = np.ones(ppc['gencost'].shape[0]) * 2
ppc['gencost'] = ppc['gencost'][:,[0,1,2,3,5,6]]
pd.DataFrame(ppc['gencost'])

,0,1,2,3,4,5
0,2.0,0.0,0.0,2.0,2.00,0.0
1,2.0,0.0,0.0,2.0,1.75,0.0
2,2.0,0.0,0.0,2.0,1.00,0.0
3,2.0,0.0,0.0,2.0,3.25,0.0
4,2.0,0.0,0.0,2.0,3.00,0.0
5,2.0,0.0,0.0,2.0,3.00,0.0


In [128]:
cost_list = []

for load in load_list:
    cost_list.append(np.array([2., 0., 0., 2., -10., 0]))
    
ppc['gencost'] = np.concatenate((ppc['gencost'], np.array(cost_list)))

In [129]:
#ppc['gencost'][:, 4] = np.zeros(ppc['gencost'].shape[0])

#ppc['gencost'][-6: , 4] = 0.
#pd.DataFrame(ppc['gencost'])

Also it contains only generation costs, to satisfy welfare cost function:

\begin{equation}
f = \sum\limits_{d \in {\cal D}} C_dP_d - \sum\limits_{g \in {\cal G}} C_gP_g = {\bf C}_d^{\top}{\bf P}_d - {\bf C}_g^{\top}{\bf P}_g\to \max
\end{equation}

From `MATPOWER` documentation the standard OPF problem takes the following form:

\begin{align}
\min_x~& f(x)\\
s.t.~&g(x) = 0\\
&h(x) \leq 0\\
x_{{\rm min}} &\leq x \leq x_{{\rm max}}
\end{align}

Then

\begin{equation}
f = {\bf C}_g^{\top}{\bf P}_g - {\bf C}_d^{\top}{\bf P}_d \to \min
\end{equation}


In [130]:
ppopt = ppoption(VERBOSE=0)
results = runopf(ppc, ppopt)


Converged in 1.16 seconds
Objective Function Value = 281.66 $/hr
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses             30     Total Gen Capacity     335.0        -170.0 to 405.9
Generators        11     On-line Capacity       335.0        -170.0 to 405.9
Committed Gens    11     Generation (actual)    149.2              89.0
Loads             16     Load                   146.3              89.5
  Fixed           16       Fixed                146.3              89.5
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             2     Shunt (inj)             -0.0               0.2
Branches          41     Losses (I^2 * Z)         2.86             14.63
Transformers       0     Branch Charging (inj)     -               15.0
Inter-ties         7     Total Inter-ti

In [131]:
#pd.DataFrame(results['bus'], columns=bus_cols)

In [132]:
#pd.DataFrame(results['gen'], columns=gen_cols)

In [133]:
ppc['gencost'][:,4][: 6] = 400
ppc['gencost'][:,4][6: ] = -420*2.5
ppc['gencost'][:,4][-2: ] = -420*2.5

In [134]:
#ppc['gencost'][0, 5] = 500.
#ppc['gencost'][1, 5] = 500.
#ppc['gencost'][3, 5] = 300.
#ppc['gencost'][2, 5] = 100.
#ppc['gencost'][4, 5] = 5.
#ppc['gencost'][5, 5] = 5.
ppc['gencost'][0, 4] = 15.
ppc['gencost'][1, 4] = 10.
ppc['gencost'][3, 4] = 15.
ppc['gencost'][2, 4] = 10.
ppc['gencost'][4, 4] = 50.
ppc['gencost'][5, 4] = 30.
ppc['gencost'][:,4:] *= 100.
#pd.DataFrame(ppc['gencost'])
ppc['gencost'][:,4] /= 2.5
ppc['gencost'][4,4] = 10000.

In [135]:
pd.DataFrame(ppc['gencost'])

,0,1,2,3,4,5
0,2.0,0.0,0.0,2.0,600.0,0.0
1,2.0,0.0,0.0,2.0,400.0,0.0
2,2.0,0.0,0.0,2.0,400.0,0.0
3,2.0,0.0,0.0,2.0,600.0,0.0
4,2.0,0.0,0.0,2.0,10000.0,0.0
5,2.0,0.0,0.0,2.0,1200.0,0.0
6,2.0,0.0,0.0,2.0,-42000.0,0.0
7,2.0,0.0,0.0,2.0,-42000.0,0.0
8,2.0,0.0,0.0,2.0,-42000.0,0.0
9,2.0,0.0,0.0,2.0,-42000.0,0.0


In [136]:
results = runopf(ppc, ppopt)


Converged in 1.16 seconds
Objective Function Value = 66442.56 $/hr
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses             30     Total Gen Capacity     335.0        -170.0 to 405.9
Generators        11     On-line Capacity       335.0        -170.0 to 405.9
Committed Gens    11     Generation (actual)    149.1              89.1
Loads             16     Load                   146.3              89.5
  Fixed           16       Fixed                146.3              89.5
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             2     Shunt (inj)             -0.0               0.2
Branches          41     Losses (I^2 * Z)         2.82             14.74
Transformers       0     Branch Charging (inj)     -               14.9
Inter-ties         7     Total Inter-

In [137]:
#pd.DataFrame(results['bus'], columns=bus_cols)
#pd.DataFrame(results['gen'], columns=gen_cols)

In [138]:
pd.DataFrame(ppc['gencost'])

,0,1,2,3,4,5
0,2.0,0.0,0.0,2.0,600.0,0.0
1,2.0,0.0,0.0,2.0,400.0,0.0
2,2.0,0.0,0.0,2.0,400.0,0.0
3,2.0,0.0,0.0,2.0,600.0,0.0
4,2.0,0.0,0.0,2.0,10000.0,0.0
5,2.0,0.0,0.0,2.0,1200.0,0.0
6,2.0,0.0,0.0,2.0,-42000.0,0.0
7,2.0,0.0,0.0,2.0,-42000.0,0.0
8,2.0,0.0,0.0,2.0,-42000.0,0.0
9,2.0,0.0,0.0,2.0,-42000.0,0.0


In [139]:
results.keys()

dict_keys(['version', 'baseMVA', 'bus', 'gen', 'branch', 'areas', 'gencost', 'order', 'om', 'x', 'mu', 'f', 'var', 'lin', 'nln', 'et', 'success', 'raw'])

In [140]:
vv, _, nn, _ = results['om'].get_idx()
vv, nn

({'i1': {'Va': 0, 'Vm': 30, 'Pg': 60, 'Qg': 71},
  'iN': {'Va': 30, 'Vm': 60, 'Pg': 71, 'Qg': 82},
  'N': {'Va': 30, 'Vm': 30, 'Pg': 11, 'Qg': 11}},
 {'i1': {'Pmis': 0, 'Qmis': 30, 'Sf': 60, 'St': 101},
  'iN': {'Pmis': 30, 'Qmis': 60, 'Sf': 101, 'St': 142},
  'N': {'Pmis': 30, 'Qmis': 30, 'Sf': 41, 'St': 41}})

In [141]:
dgT, dgeqT = get_Jac(ppc, ppopt, results)
muls = get_muls(results)
is_price_forming(results['gen'], results['gencost'], verbose=True)

Price forming bus #2	 C_g=400.000000
Price forming bus #22	 C_g=400.000000
Price forming bus #27	 C_g=600.000000


[1, 21, 26]

\begin{equation}
\mu = 
\begin{pmatrix}
{\bf 0}\\
\mu^{\max} - \mu^{\min}
\end{pmatrix}
\end{equation}


$$\sigma = (\sigma_{from}, \sigma_{to})^{\top}$$
as according to the original code (opf_consfcn)
$${\bf J}_S^T = ({\bf J}_{S~from}, {\bf J}_{S~to})$$

In [142]:
sigma = np.concatenate((muls['mu_s_from'], muls['mu_s_to']))

Let's obtain FOC for our OPF

\begin{align}
&\left(\frac{\partial L}{\partial {\bf X}}\right)^{\top} = {\bf J}^{\top}{\lambda} + {\bf J}_S^{\top}{\sigma} + {\mu} = 0\\
&\left(\frac{\partial L}{\partial {\bf P}_g}\right)^{\top} = {\bf C}_g - { \lambda}^P + {\pi}^{\max} - {\pi}^{\min} = 0,\\
&\left(\frac{\partial L}{\partial {\bf P}_d}\right)^{\top} = -{\bf C}_d + { \lambda}^P + {\psi}^{\max} - {\psi}^{\min} = 0,\\
&\left(\frac{\partial L}{\partial {\bf Q}_g}\right)^{\top} = {\bf 0} - { \lambda}^Q + {\rho}^{\max} - { \rho}^{\min} = 0
\end{align}

From here we should fix ${\lambda}^Q$ as we don't want to change ${\rho}^{\max}, { \rho}^{\min}$ and following we should recompute costs after obtaining new ${ \lambda}^P$ with ${\pi}^{\max}, {\pi}^{\min}, {\psi}^{\max}, {\psi}^{\min}$ from original solution.

$${\bf J}^T
\begin{bmatrix}
\lambda^P\\
\lambda^Q
\end{bmatrix}
=
\begin{bmatrix}
{\bf J}_P^T & {\bf J}_Q^T 
\end{bmatrix}
\begin{bmatrix}
\lambda^P\\
\lambda^Q
\end{bmatrix}
=
{\bf J}_P^T\lambda^P + {\bf J}_Q^T\lambda^Q
=
{\bf J}_P^T\begin{bmatrix}
\lambda_{ok}^P\\
\lambda_{form}^P
\end{bmatrix} + {\bf J}_Q^T\lambda^Q
=
{\bf J}_{P~ok}^T\lambda_{ok}^P + {\bf J}_{P~form}^T\lambda_{form}^P + {\bf J}_Q^T\lambda^Q
$$


$$
{\bf J}_{P~ok}^T\lambda_{ok}^P + {\bf J}_{P~form}^T\lambda_{form}^P + {\bf J}_Q^T\lambda^Q + {\bf J}_S^{T}\sigma + \begin{bmatrix}
{\bf O} & {\bf O}\\
{\bf I} & -{\bf I}
\end{bmatrix}
\begin{bmatrix}
\mu^{\max}\\
\mu^{\min}
\end{bmatrix}
 = 0
$$

This term ${\bf J}_{P~form}^T\lambda_{form}^P$ contains criminals and we just replace somehow them obtaining $\lambda_{form}^{P~new}$

$${\bf J}_{P~ok}^T\lambda_{ok}^P + {\bf J}_S^{T}\sigma + 
\begin{bmatrix}
{\bf O} & {\bf O}\\
{\bf I} & -{\bf I}
\end{bmatrix}
\begin{bmatrix}
\mu^{\max}\\
\mu^{\min}
\end{bmatrix} = - {\bf J}_Q^T\lambda^Q - {\bf J}_{P~form}^T\lambda_{form}^{P~new}$$

$${\bf J}_{\mu} = 
\begin{bmatrix}
{\bf O} & {\bf O}\\
{\bf I} & -{\bf I}
\end{bmatrix}~~
\mu = 
\begin{bmatrix}
\mu^{\max}\\
\mu^{\min}
\end{bmatrix}
$$

$$
\begin{bmatrix}
{\bf J}_{P~ok}^T & {\bf J}_S^{T} & {\bf J}_{\mu}
\end{bmatrix}
\begin{bmatrix}
\lambda_{ok}^P\\
\sigma\\
\mu
\end{bmatrix}
= - {\bf J}_Q^T\lambda^Q - {\bf J}_{P~form}^T\lambda_{form}^{P~new}
$$

$${\bf A}{\bf x} = {\bf b}$$

If $\sigma^{old} = 0 \Rightarrow \sigma^{new} = 0$ let's denote corresponding sub-matrices ${\bf J}_S^{\top}[:, I\{\sigma^{old} > 0\}]$

And the same for $\mu^{old}, \mu^{new}$

Then
$${\bf A} = 
\begin{bmatrix}
{\bf J}_{P~ok}^T & {\bf J}_S^{T}[:, I\{\sigma^{old} > 0\}] & {\bf J}_{\mu}[:, I\{\mu^{old} > 0\}]
\end{bmatrix}
$$
$$
{\bf x} = 
\begin{bmatrix}
\lambda_{ok}^P\\
\sigma[I\{\sigma^{old} > 0\}]\\
\mu[I\{\mu^{old} > 0\}]
\end{bmatrix}
$$

This system we will solve in terms of LS:

\begin{align}
\min &\|{\bf A}{\bf x} - {\bf b}\|_2^2\\
s.t.~&{\bf x} \geq 0
\end{align}

Constraints could be user defined e.g. market constraints (#5 from list on the doc).

Also we could obtain  $\lambda^P$ from FOC and solve the frobplem with a bit different unknown ${\bf x} = ((\pi^{\max})^{\top}, (\psi^{\max})^{\top}, (\pi^{\min})^{\top}, (\psi^{\min})^{\top}, \sigma^{\top}, \mu^{\top})^{\top}$

Let ${\bf c} = ({\bf C}_g^{\top}, -{\bf C}_d^{\top})^{\top}$ then $\lambda^P = {\rm sign}({\bf c})\cdot({\bf c} + \gamma^{\max} - \gamma^{\min})$, $\gamma = ((\pi^{\max})^{\top}, (\psi^{\max})^{\top})^{\top}$ and ${\rm sign}(\cdot)$ is elementwise function and ${\bf a} \cdot {\bf b}$ is the Hadamard product.

In [143]:
idx_sigma = sigma > 0
muls.keys()
#так как у нас лоады это отрицательные генераторы
#надо нормально написать пересчет костов
#pi_psi = muls['mu_p_max'] - muls['mu_p_min']

dict_keys(['mu_v_max', 'mu_v_min', 'mu_s_from', 'mu_s_to', 'lam_p', 'lam_q', 'mu_p_max', 'mu_p_min', 'mu_q_max', 'mu_q_min', 'C'])

In [144]:
ppc_2hrs = generate_Nhrs_cases(ppc, 2)

In [145]:
#pd.DataFrame(ppc['bus'], columns=bus_cols)

In [146]:
ppc_2hrs = set_hours_constr(ppc_2hrs, (-1e-4, 15.), (-50., 50.), Nhrs=2)

In [147]:
results_2hrs = runopf(ppc_2hrs)


opf_setup: Warning: Multiple reference buses.
           For a system with islands, a reference bus in each island
           may help convergence, but in a fully connected system such
           a situation is probably not reasonable.

PYPOWER Version 5.1.4, 27-June-2018 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011

Numerically Failed

Did not converge in 1 iterations.

Did not converge (0.10 seconds)

Objective Function Value = 6781021.00 $/hr
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses             60     Total Gen Capacity     670.0        -340.0 to 811.8
Generators        22     On-line Capacity       670.0        -340.0 to 811.8
Committed Gens    22     Generation (actual)    185.0             235.9
Loads             32     Load                

/home/rectangle/.local/lib/python3.6/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:193: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)


In [148]:
muls_2hrs = get_muls(results_2hrs, Nhrs=2)
muls_2hrs['us_int']


array([10000., 10000.,     0.,     0.,     0., 10000., 10000.,     0.,
           0.,     0.,     0.,     0., 10000.,     0., 10000.,     0.,
           0.,     0., 10000.,     0.,     0., 10000., 10000.,     0.,
           0.,     0., 10000.,     0., 10000.,     0.])

In [149]:
is_price_forming(ppc_2hrs['gen'], ppc_2hrs['gencost'], Nhrs=2, results=results_2hrs, verbose=True)

Hour 1:Price forming bus #1	 C_g=600.000000
Hour 1:Price forming bus #2	 C_g=400.000000
Hour 1:Price forming bus #22	 C_g=400.000000
Hour 1:Price forming bus #27	 C_g=600.000000
Hour 1:Price forming bus #23	 C_g=10000.000000
Hour 1:Price forming bus #13	 C_g=1200.000000
Hour 2:Price forming bus #22	 C_g=400.000000
Hour 2:Price forming bus #27	 C_g=600.000000
Hour 2:Price forming bus #23	 C_g=10000.000000


[0, 1, 21, 26, 22, 12, 51, 56, 52]